In [13]:
print("Hello World")

Hello World

In [ ]:
gap = GAP("Instances/gap1.txt", 0)

variables = ["r", "c", "b", "m", "t"]
values = [gap.r, gap.c, gap.b, gap.m, gap.t]

for i in 1:length(variables)
    println("$(variables[i]) : $(values[i])")
end

r : [23 10 15 13 17 10 13 6 100 21 20 100 100 25 100; 17 13 8 23 11 18 7 22 13 5 24 24 15 10 100; 22 17 22 23 20 11 17 25 23 9 22 20 15 9 25; 5 19 25 16 15 10 18 9 11 20 7 21 15 8 25; 22 9 10 23 19 21 17 15 15 17 25 19 10 9 100]
c : [22 21 16 17 21 15 17 22 22 25 18 20 24 15 22; 23 24 19 15 16 21 15 25 16 21 20 19 16 23 20; 21 20 21 25 21 20 21 19 17 16 25 19 15 15 15; 17 21 25 25 23 22 20 19 20 25 15 20 21 25 23; 15 25 23 19 17 17 25 24 24 17 24 19 18 19 16]
b : [32, 37, 44, 35, 40]
m : 5
t : 15


In [ ]:
# r : quantité de ressources consommées par tâche (matrix)
gap.r

5×15 Matrix{Any}:
 23  10  15  13  17  10  13   6   9  21  20   7   9  25   8
 17  13   8  23  11  18   7  22  13   5  24  24  15  10  22
 22  17  22  23  20  11  17  25  23   9  22  20  15   9  25
  5  19  25  16  15  10  18   9  11  20   7  21  15   8  25
 22   9  10  23  19  21  17  15  15  17  25  19  10   9  21

In [ ]:
# c : gain par tâche pour chaque agent (matrix)
gap.c

5×15 Matrix{Any}:
 22  21  16  17  21  15  17  22  22  25  18  20  24  15  22
 23  24  19  15  16  21  15  25  16  21  20  19  16  23  20
 21  20  21  25  21  20  21  19  17  16  25  19  15  15  15
 17  21  25  25  23  22  20  19  20  25  15  20  21  25  23
 15  25  23  19  17  17  25  24  24  17  24  19  18  19  16

In [ ]:
# b : quantité de ressource de l'agent i (vect)
gap.b

5-element Vector{Any}:
 32
 37
 44
 35
 40

In [ ]:
# m : nombre d'agents (int) --> nombre de lignes
gap.m

5

In [ ]:
# t : nombre de tâches (int) --> nombre de colonnes
gap.t

15

### Heuristique gloutonne

### Create CSV

In [ ]:
include("create_csv.jl")
# create_csv()

hill_climbing! (generic function with 1 method)

### Voisinages

In [58]:
assignment

20-element Vector{Int64}:
 3
 1
 1
 2
 5
 2
 3
 2
 5
 3
 2
 4
 4
 4
 1
 3
 5
 4
 1
 5

In [49]:
is_feasible(assignment, "Instances/gap2.txt", 2)

true

In [50]:
is_feasible(swap_tasks(assignment, 1, 2), "Instances/gap2.txt", 2)

false

### Recuit

In [102]:
include("Neighbours.jl")
using Random

function recuit(x0, filename, id, mu, T0, iter_max)

    count = 0

    x_max = copy(x0)
    x = copy(x0)
    T = T0

    while T > 1
        for iter in 1:iter_max
            i = rand(1:m)
            j = rand(1:m)
            while j == i
                j = rand(1:m)
            end
            x_p = swap_tasks(x, i, j)
            
            if is_feasible(x_p, filename, id)
                delta = cal_cost(x_p, filename, id) - cal_cost(x, filename, id)
                if delta > 0 # Maximisation (< 0 pour une minimisation)
                    # println("better")
                    x = copy(x_p)
                    if cal_cost(x, filename, id) > cal_cost(x_max, filename, id)
                        print("better")
                        x_max = copy(x)
                    end
                else
                    q = rand()
                    if q <= exp(-delta/T)
                        x = x_p
                    end
                end
            end
        end
        T = mu * T
        count += 1
        println(T)
    end
    return x
end

recuit (generic function with 2 methods)

In [125]:
include("Gap_struc.jl")

recuit (generic function with 1 method)

In [121]:
gap = GAP("Instances/gap2.txt", 2)

GAP([18 6 … 5 13; 12 16 … 12 18; … ; 6 23 … 24 13; 16 10 … 8 9], [11 19 … 18 14; 13 16 … 17 15; … ; 17 12 … 12 11; 19 22 … 13 13], [45, 51, 48, 45, 46], 5, 20, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [122]:
find_greedy_solution!(gap)

In [123]:
gap.task_assignation

20-element Vector{Int64}:
 4
 1
 1
 3
 4
 2
 3
 2
 5
 5
 2
 4
 3
 4
 1
 1
 1
 4
 1
 5

In [126]:
x = recuit(gap, 0.95, 100, 1000)

UndefVarError: UndefVarError: `swap_tasks` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [104]:
is_feasible(x, "Instances/gap2.txt", 2)

true

In [105]:
cal_cost(x, "Instances/gap2.txt", 2)

377

In [106]:
greedy("Instances/gap2.txt", 2)

([3, 1, 1, 2, 5, 2, 3, 2, 5, 3, 2, 4, 4, 4, 1, 3, 5, 4, 1, 5], 358)

In [99]:
cost

358

### Heuristique gloutonne

In [36]:
ratio = c./r

5×15 Matrix{Float64}:
 0.956522  2.1      1.06667   1.30769   …  2.66667  0.6      2.75
 1.35294   1.84615  2.375     0.652174     1.06667  2.3      0.909091
 0.954545  1.17647  0.954545  1.08696      1.0      1.66667  0.6
 3.4       1.10526  1.0       1.5625       1.4      3.125    0.92
 0.681818  2.77778  2.3       0.826087     1.8      2.11111  0.761905

In [37]:
argmax(ratio)

CartesianIndex(2, 10)

In [38]:
current_ressource = zeros(m)

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [39]:
current_ressource[1] < b[1]

true

In [40]:
ratio[10,:]

BoundsError: BoundsError: attempt to access 5×15 Matrix{Float64} at index [10, 1:15]

In [41]:
ratio

5×15 Matrix{Float64}:
 0.956522  2.1      1.06667   1.30769   …  2.66667  0.6      2.75
 1.35294   1.84615  2.375     0.652174     1.06667  2.3      0.909091
 0.954545  1.17647  0.954545  1.08696      1.0      1.66667  0.6
 3.4       1.10526  1.0       1.5625       1.4      3.125    0.92
 0.681818  2.77778  2.3       0.826087     1.8      2.11111  0.761905